In [1]:
# 在 Colab 第一个代码块，运行一次即可
!pip install -q "transformers>=4.39" ctransformers sentence-transformers \
                 langchain langchain-community chromadb rank_bm25

In [2]:
# === 块 2：全局配置（修改这里） ===
DATA_PATH = "item_desc_dataset.txt"   # TSV 文件，每行：标题 \t 描述

# 样本量：设为 0 或负数，或大于总行数时，读取全部；否则随机抽 SAMPLE_SIZE 行
SAMPLE_SIZE = 1000     # ← 在 0～2129187 之间修改

# 用户只需修改引号里的关键词，就能改变检索输入
QUERY = "夹克"   # ← 修改这里


In [3]:
# === 块 3：导入所有依赖 ===
import os, random, shutil, textwrap, jieba
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers import BM25Retriever
from transformers import AutoTokenizer, AutoModelForCausalLM, logging as hf_logging
from rank_bm25 import BM25Okapi

hf_logging.set_verbosity_error()  # 关闭 HF 过多日志


In [4]:
# === 块 4：加载 & 抽样数据 ===
# 读取所有行
with open(DATA_PATH, "r", encoding="utf-8", errors="replace") as f:
    lines = f.readlines()
# 随机抽样逻辑
random.seed(42)
if SAMPLE_SIZE > 0 and len(lines) > SAMPLE_SIZE:
    lines = random.sample(lines, SAMPLE_SIZE)

# 构建 Document 列表
docs = []
for line in lines:
    if "\t" not in line:
        continue
    title, desc = line.strip().split("\t", 1)
    docs.append(Document(page_content=f"{title}\n{desc}"))

print(f"[Step 1] 读取并抽样数据 → 共 {len(docs)} 条记录")


[Step 1] 读取并抽样数据 → 共 1000 条记录


In [5]:
# === 块 5：构建向量库 & BM25 索引 ===
# 文本切块
splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# 加载嵌入模型
embed_model = "BAAI/bge-small-zh-v1.5"
print(f"[Step 2] 加载嵌入模型: {embed_model}")
embeddings = HuggingFaceEmbeddings(model_name=embed_model)

# 清理&创建持久化目录
persist_dir = "./chroma_db"
if os.path.isdir(persist_dir):
    shutil.rmtree(persist_dir)
os.makedirs(persist_dir, exist_ok=True)

# 构建 Chroma 向量库
vectordb = Chroma.from_documents(
    chunks, embeddings, persist_directory=persist_dir
)

# 构建 BM25 检索器
bm25 = BM25Retriever.from_documents(chunks)
bm25.k = 5

print(f"[Step 2] 已构建向量库 & BM25，共切分出 {len(chunks)} 段文本")






[Step 2] 加载嵌入模型: BAAI/bge-small-zh-v1.5


<ipython-input-5-3bc6b42b9862>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embed_model)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[Step 2] 已构建向量库 & BM25，共切分出 1000 段文本


In [6]:
# === 块 6：加载 Qwen3-0.6B 模型 ===
model_name = "Qwen/Qwen3-0.6B"
print(f"[Step 3] 加载 LLM: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name, trust_remote_code=True, device_map="auto"
)
print("[Step 3] 模型就绪")


[Step 3] 加载 LLM: Qwen/Qwen3-0.6B
[Step 3] 模型就绪


In [7]:
# === 改进后的混合检索代码片段 ===

# 1. 先用 jieba 对所有 chunks 分词，准备 BM25
texts = [chunk.page_content for chunk in chunks]
tokenized_corpus = [list(jieba.cut(t)) for t in texts]
bm25_okapi = BM25Okapi(tokenized_corpus)

# 2. BM25 检索并归一化得分（分数越高越相关）
query_tokens = list(jieba.cut(QUERY))
bm25_raw_scores = bm25_okapi.get_scores(query_tokens)
# 归一化：score / max_score
max_bm = max(bm25_raw_scores) or 1.0
bm25_scores = [s / max_bm for s in bm25_raw_scores]

# 3. 向量检索并归一化得分（Chroma 返回的是距离，越小越相关）
vec_results = vectordb.similarity_search_with_score(QUERY, k=5)
# similarity_search_with_score 返回 [(doc, distance), ...]
# 转换成“相关度分数”：score = 1 / (1 + distance)
vec_scores = { d.page_content: 1/(1+dist) for d, dist in vec_results }

# 4. 加权融合：这里简单平均，也可以用别的权重
combined_scores = {}
for idx, doc in enumerate(chunks):
    text = doc.page_content
    combined_scores[text] = bm25_scores[idx]  # BM25 部分

for doc, sim_score in vec_scores.items():
    combined_scores[doc] = combined_scores.get(doc, 0) + sim_score

# 5. 排序 & 取 top5
sorted_docs = sorted(
    combined_scores.items(),
    key=lambda kv: kv[1],
    reverse=True
)
top5 = [Document(page_content=txt) for txt, _ in sorted_docs[:5]]

# 6. 打印结果
print(f"[Step 4] 混合检索后 Top 5：")
for i, d in enumerate(top5, 1):
    snippet = d.page_content.replace("\n", " ")[:200]
    print(f"[{i}] {snippet}…")


Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.985 seconds.
DEBUG:jieba:Loading model cost 0.985 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


[Step 4] 混合检索后 Top 5：
[1] dumb秋季新款刺绣夹克ulzzang男士外套港风bf宽松上衣学生外衣潮 dumb秋冬加绒刺绣夹克ulzzang男士外套港风bf宽松上衣学生外衣潮。90后懒人显瘦潮流都市雅痞约会工作撩妹夹克…
[2] 千纸鹤夹克2018秋冬季男士青年商务纯色外套飞行休闲夹克潮男3257 时尚潮流资讯型男撞色潮流夹克，优质的面料具有很好的防紫外线和吸湿排汗的功效，炫酷的外表，宽松的版型，显得更加随心，独特的设计，更显时尚、潮流，搭配深色休闲裤和时尚休闲鞋，成熟之中透露着活力…
[3] 男士外套2018新款春秋冬季夹克韩版帅气男装秋装秋冬加绒加厚上衣 大热军事风型男复古潮流夹克，全棉材质，舒适而又富有弹性，款式新颖，举手投足间尽显绅士气度，时尚的设计，细腻的做工，你值得拥有…
[4] 七匹狼夹克男2018秋冬季新款棒球服男士外套中年爸爸休闲上衣1808 型男原创印花潮流大叔夹克，质感面料，穿出时尚潮流气质，精致做工，上身舒适，毫无束缚感，采用简约而不简单的设计，十分的休闲百搭，时尚百搭，适合各种身材…
[5] 布衣传说连帽中长款夹克男装秋季韩版修身青年外套上衣服潮流褂子 型男学生原创街拍男士秋季外套，面料很舒服，穿着显身材，做工精细美观，凸显优雅纯品，精致的做工，实用性与美观性完美结合，上身显得身材比例超好，低调中带着酷劲…


In [ ]:
# === 块 8：基于检索结果，生成 3 组【标题 + 描述】 ===
context = "\n\n".join(d.page_content for d in docs)

context

QUERY

prompt = textwrap.dedent(f"""
你是一位精通电商爆款文案的资深运营。根据检索到的商品资料，为【{QUERY}】生成3组差异化【标题 + 产品描述】，需严格遵循：

【标题规则】
28-30字无标点关键词组合，必须包含：
季节属性 | 核心痛点 | 材质工艺 | 人群定位 | 风格元素
（参考案例结构："秋冬1000d加绒加厚微压连裤袜显瘦美腿打底袜女哑光保暖神器"）

【描述规则】
80-100字分4个模块：
1. 工艺细节（原料/技术/专利）
2. 场景痛点（保暖/显瘦/防勾丝）
3. 对比优势（与同类产品差异化）
4. 情感价值（自信/魅力/舒适体验）
（参考："高密度天鹅绒叠加微压技术显瘦不紧绷脚踝薄绒处理搭配单鞋不臃肿..."）

【格式要求】
严格使用数字编号+缩进格式，样例：
1. 标题：秋冬加厚300g磨毛工装裤男束脚小脚裤冬季防风加绒休闲裤韩版潮流
   描述：采用300g海岛棉磨毛工艺锁温效果提升40%,零下15℃单裤过冬告别臃肿叠穿,膝盖立体裁片比普通工装裤活动更自如,做旧金属扣与抽绳设计打造高街潮男气质

参考检索内容：
{context}
""").strip()

# 调用模型
messages = [{"role": "user", "content": prompt}]
text     = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs   = tokenizer(text, return_tensors="pt").to(model.device)
out_ids  = model.generate(**inputs, max_new_tokens=512)[0][len(inputs.input_ids[0]):]
result   = tokenizer.decode(out_ids, skip_special_tokens=True)

print("\n[Step 5] 生成文案结果：\n")
print(result)
